In [2]:
import os
import cv2
import numpy as np

In [31]:
# Define a function to compute Motion History Images (MHIs)
def compute_mhi(video_path, duration=1.0, threshold=64):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Calculate the max value for MHI based on duration and FPS of the video
    mhi_duration = int(fps * duration)  # Duration in frames
    timestamp = 0  # Current time stamp
    mhi = None  # Motion History Image
    
    # Read the first frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return None
    # Convert to grayscale
    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Loop through the video
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Compute the absolute difference between the current frame and the previous frame
        frame_diff = cv2.absdiff(current_frame, prev_frame)
        # Threshold the difference
        _, motion_mask = cv2.threshold(frame_diff, threshold, 1, cv2.THRESH_BINARY)
        timestamp += 1  # Update the timestamp
        
        # Update MHI
        if mhi is None:
            mhi = np.zeros_like(prev_frame, dtype=np.float32)
        # Decrease the MHI value for all pixels by 1 to a minimum of 0
        mhi = np.maximum(mhi - 1/mhi_duration, 0)
        # Where there is motion, set the MHI to the maximum (current timestamp)
        mhi[motion_mask == 1] = timestamp

        prev_frame = current_frame

    # Normalize the MHI to the range of 0-255 for visualization
    mhi = np.uint8(np.clip((mhi - (timestamp - mhi_duration)) / mhi_duration, 0, 1) * 255)
    cap.release()
    return mhi

In [43]:

# Path to the folder containing videos
video_folder = r'/home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_random_video'

# List all files in the folder
video_files = os.listdir(video_folder)

# Iterate over video files
for video_file in video_files:
    # Check if the file is an MKV video
    if video_file.endswith('.mkv'):
        video_path = os.path.join(video_folder, video_file)
        # Compute the MHI for the provided video
        mhi_image = compute_mhi(video_path)
        
        # Check if MHI was successfully computed
        if mhi_image is not None:
            # Create a folder named 'mhi' if it doesn't exist
            mhi_folder = os.path.join(video_folder, 'mhi')
            if not os.path.exists(mhi_folder):
                os.makedirs(mhi_folder)
            
            # Save the resulting MHI image to the 'mhi' folder
            mhi_filename = f'mhi_{os.path.splitext(video_file)[0]}.png'
            mhi_path = os.path.join(mhi_folder, mhi_filename)
            cv2.imwrite(mhi_path, mhi_image)
            print(f"MHI saved: {mhi_path}")
        else:
            print("Failed to compute MHI for", video_file)

MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_random_video/mhi/mhi_random_video_1_cam_80.png
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_random_video/mhi/mhi_random_video_1_cam_34.png
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_random_video/mhi/mhi_random_video_1_cam_123.png
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_random_video/mhi/mhi_random_video_1_cam_198.png
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_random_video/mhi/mhi_random_video_1_cam_78.png
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_1/3th_court_1_cam_08_02_random_video/mhi/mhi_random_video_1_cam_92.png
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_202

In [14]:
import cv2
import numpy as np

# Define a function to compute Motion History Images (MHIs) with adjustable threshold
def compute_mhi_with_adjustable_threshold(video_path, duration=1.0, initial_threshold=64):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Calculate the max value for MHI based on duration and FPS of the video
    mhi_duration = int(fps * duration)  # Duration in frames
    timestamp = 0  # Current time stamp
    mhi = None  # Motion History Image
    
    # Read the first frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return None
    
    # Convert to grayscale
    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Loop through the video
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Compute the absolute difference between the current frame and the previous frame
        frame_diff = cv2.absdiff(current_frame, prev_frame)
        
        # Apply adjustable threshold
        _, motion_mask = cv2.threshold(frame_diff, initial_threshold, 1, cv2.THRESH_BINARY)
        
        timestamp += 1  # Update the timestamp
        
        # Update MHI
        if mhi is None:
            mhi = np.zeros_like(prev_frame, dtype=np.float32)
        # Decrease the MHI value for all pixels by 1 to a minimum of 0
        mhi = np.maximum(mhi - 1/mhi_duration, 0)
        # Where there is motion, set the MHI to the maximum (current timestamp)
        mhi[motion_mask == 1] = timestamp

        prev_frame = current_frame

    # Normalize the MHI to the range of 0-255 for visualization
    mhi = np.uint8(np.clip((mhi - (timestamp - mhi_duration)) / mhi_duration, 0, 1) * 255)
    cap.release()
    return mhi

# Adjust these parameters accordingly
video_path = "/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_rotation_scoreboard/rotation_scoreboad_live_cam_146.mkv"
initial_threshold = 160

# Compute MHI with adjustable threshold
# mhi = compute_mhi_with_adjustable_threshold(video_path, initial_threshold=initial_threshold)


# Path to the folder containing videos
video_folder = r'/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_rotation_scoreboard/'

# List all files in the folder
video_files = os.listdir(video_folder)

# Iterate over video files
for video_file in video_files:
    # Check if the file is an MKV video
    if video_file.endswith('.mkv'):
        video_path = os.path.join(video_folder, video_file)
        # Compute the MHI for the provided video
        mhi_image = compute_mhi_with_adjustable_threshold(video_path, initial_threshold=initial_threshold)
        
        # Check if MHI was successfully computed
        if mhi_image is not None:
            # Create a folder named 'mhi' if it doesn't exist
            mhi_folder = os.path.join(video_folder, 'mhi')
            if not os.path.exists(mhi_folder):
                os.makedirs(mhi_folder)
            
            # Save the resulting MHI image to the 'mhi' folder
            mhi_filename = f'mhi_12_{os.path.splitext(video_file)[0]}.png'
            mhi_path = os.path.join(mhi_folder, mhi_filename)
            cv2.imwrite(mhi_path, mhi_image)
            print(f"MHI saved: {mhi_path}")
        else:
            print("Failed to compute MHI for", video_file)
# Display the resulting MHI


MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_rotation_scoreboard/mhi/mhi_160_rotation_scoreboad_live_cam_146.png


### Median 

In [18]:
import cv2
import numpy as np

# Medyan filtreleme uygulayarak Arka Plan Gürültüsünü Azaltma
def medyan_filtreleme(image, kernel_size=3):
    return cv2.medianBlur(image, kernel_size)

# Hareket Tarih Görüntüleri (MHIs) hesaplamak için bir fonksiyon tanımla
def hesapla_mhi(video_yolu, sure=1.0, esik_degeri=64):
    # Video dosyasını aç
    cap = cv2.VideoCapture(video_yolu)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # MHI için maksimum değeri video süresi ve FPS kullanarak hesapla
    mhi_suresi = int(fps * sure)  # Kare cinsinden süre
    zaman_damgasi = 0  # Mevcut zaman damgası
    mhi = None  # Hareket Tarih Görüntüsü
    
    # İlk kareyi oku
    ret, cerceve = cap.read()
    if not ret:
        print("Video okunamadı")
        return None
    
    # Grayscale'e dönüştür
    onceki_cerceve = cv2.cvtColor(cerceve, cv2.COLOR_BGR2GRAY)
    # Videoyu döngüye al
    while True:
        ret, cerceve = cap.read()
        if not ret:
            break
        mevcut_cerceve = cv2.cvtColor(cerceve, cv2.COLOR_BGR2GRAY)
        # Mevcut kare ile önceki kare arasındaki mutlak farkı hesapla
        cerceve_farki = cv2.absdiff(mevcut_cerceve, onceki_cerceve)
        
        # Eşik uygula
        _, hareket_maske = cv2.threshold(cerceve_farki, esik_degeri, 1, cv2.THRESH_BINARY)
        
        zaman_damgasi += 1  # Zaman damgasını güncelle
        
        # MHI'yi güncelle
        if mhi is None:
            mhi = np.zeros_like(onceki_cerceve, dtype=np.float32)
        # Tüm pikseller için MHI değerini 1'e, minimum 0'a kadar azalt
        mhi = np.maximum(mhi - 1/mhi_suresi, 0)
        # Hareket olan piksellerde MHI'yi maksimuma (mevcut zaman damgası) ayarla
        mhi[hareket_maske == 1] = zaman_damgasi

        onceki_cerceve = mevcut_cerceve

    # Arka plan gürültüsünü azaltmak için medyan filtreleme uygula
    mhi = medyan_filtreleme(mhi)

    # MHI'yi görselleştirmek için 0-255 aralığına normalize et
    mhi = np.uint8(np.clip((mhi - (zaman_damgasi - mhi_suresi)) / mhi_suresi, 0, 1) * 255)
    cap.release()
    return mhi

# Bu parametreleri isteğe bağlı olarak ayarlayabilirsiniz
esik_degeri = 32  # Eşik değeri ayarlayın
sure = 2.0

# MHI'yi hesapla ve arka plan gürültüsünü azalt
#mhi = hesapla_mhi(video_yolu, sure=sure, esik_degeri=esik_degeri)


# Path to the folder containing videos
video_folder = r'/home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_rotation_scoreboard/'
0
# List all files in the folder
video_files = os.listdir(video_folder)

# Iterate over video files
for video_file in video_files:
    # Check if the file is an MKV video
    if video_file.endswith('.mkv'):
        video_path = os.path.join(video_folder, video_file)
        # Compute the MHI for the provided video
        mhi_image = hesapla_mhi(video_path, sure=sure, esik_degeri=esik_degeri)
        
        # Check if MHI was successfully computed
        if mhi_image is not None:
            # Create a folder named 'mhi' if it doesn't exist
            mhi_folder = os.path.join(video_folder, 'mhi_median')
            if not os.path.exists(mhi_folder):
                os.makedirs(mhi_folder)
            
            # Save the resulting MHI image to the 'mhi' folder
            mhi_filename = f'mhi_median_32_{os.path.splitext(video_file)[0]}.png'
            mhi_path = os.path.join(mhi_folder, mhi_filename)
            cv2.imwrite(mhi_path, mhi_image)
            print(f"MHI saved: {mhi_path}")
        else:
            print("Failed to compute MHI for", video_file)
# Display the resulting MHI



MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/02_13_2024/3th_court/cam_live/3th_court_live_cam_02_13_rotation_scoreboard/mhi/mhi_median_32_rotation_scoreboad_live_cam_146.png


## Arkaplan maskeleme

In [21]:
import cv2
import numpy as np
import os

# Define a function to compute Motion History Images (MHIs) with adjustable threshold
def compute_mhi_with_adjustable_threshold(video_path, arka_plan_modeli, duration=1.0, initial_threshold=64):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Calculate the max value for MHI based on duration and FPS of the video
    mhi_duration = int(fps * duration)  # Duration in frames
    timestamp = 0  # Current time stamp
    mhi = None  # Motion History Image
    
    # Read the first frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to read video")
        return None
    
    # Convert to grayscale
    prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Loop through the video
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Compute the absolute difference between the current frame and the previous frame
        frame_diff = cv2.absdiff(current_frame, prev_frame)
        
        # Apply adjustable threshold
        _, motion_mask = cv2.threshold(frame_diff, initial_threshold, 1, cv2.THRESH_BINARY)
        
        timestamp += 1  # Update the timestamp
        
        # Update MHI
        if mhi is None:
            mhi = np.zeros_like(prev_frame, dtype=np.float32)
        # Decrease the MHI value for all pixels by 1 to a minimum of 0
        mhi = np.maximum(mhi - 1/mhi_duration, 0)
        # Where there is motion, set the MHI to the maximum (current timestamp)
        mhi[motion_mask == 1] = timestamp

        prev_frame = current_frame

    # Normalize the MHI to the range of 0-255 for visualization
    mhi = np.uint8(np.clip((mhi - (timestamp - mhi_duration)) / mhi_duration, 0, 1) * 255)
    cap.release()
    return mhi

# Define a function to create the background model using multiple reference images
def arka_plan_modelini_olustur_coklu(referans_klasoru):
    # Initialize an empty list to store reference images
    referans_goruntuleri = []

    # List all files in the reference folder
    for dosya in os.listdir(referans_klasoru):
        dosya_yolu = os.path.join(referans_klasoru, dosya)
        # Read each reference image and append it to the list
        referans_goruntu = cv2.imread(dosya_yolu, cv2.IMREAD_GRAYSCALE)
        if referans_goruntu is not None:
            referans_goruntuleri.append(referans_goruntu)
        else:
            print(f"{dosya} isimli dosya okunamadı.")

    # If there are no reference images, return None
    if len(referans_goruntuleri) == 0:
        print("No reference images found.")
        return None

    # Compute the background model by taking the median of all reference images
    arka_plan_modeli = np.median(referans_goruntuleri, axis=0).astype(np.uint8)

    return arka_plan_modeli


# Adjust these parameters accordingly
video_folder = r'/home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_2/3th_court_2_cam_08_02_rotation_scoreboard'
# Adjust these parameters accordingly
referans_klasoru = "/home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_2/referans/"

# Create the background model using multiple reference images
arka_plan_modeli_coklu = arka_plan_modelini_olustur_coklu(referans_klasoru)
initial_threshold = 32



# Check if the background model was successfully created
if arka_plan_modeli_coklu is not None:
    print("Arka plan modeli başarıyla oluşturuldu.")
    
    # List all files in the folder
    video_files = os.listdir(video_folder)

    # Iterate over video files
    for video_file in video_files:
        # Check if the file is an MKV video
        if video_file.endswith('.mkv'):
            video_path = os.path.join(video_folder, video_file)
            # Compute the MHI for the provided video
            mhi_image = compute_mhi_with_adjustable_threshold(video_path, arka_plan_modeli_coklu, initial_threshold=initial_threshold)
            
            # Check if MHI was successfully computed
            if mhi_image is not None:
                # Create a folder named 'mhi' if it doesn't exist
                mhi_folder = os.path.join(video_folder, 'mhi_bgk')
                if not os.path.exists(mhi_folder):
                    os.makedirs(mhi_folder)
                
                # Save the resulting MHI image to the 'mhi' folder
                mhi_filename = f'mhi_32_bgk_{os.path.splitext(video_file)[0]}.png'
                mhi_path = os.path.join(mhi_folder, mhi_filename)
                cv2.imwrite(mhi_path, mhi_image)
                print(f"MHI saved: {mhi_path}")
            else:
                print("Failed to compute MHI for", video_file)
else:
    print("Arka plan modeli oluşturulamadı.")


Arka plan modeli başarıyla oluşturuldu.
MHI saved: /home/muhlabws4/Turkish_Open_2024_Antalya/Konya/08_02_2024/3th_court/cam_2/3th_court_2_cam_08_02_rotation_scoreboard/mhi_bgk/mhi_32_bgk_rotation_scoreboad_2_cam_49.png
